In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os

In [4]:
from IPython import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

With Seed

In [5]:
tf.set_random_seed(123)

In [6]:
tf.reset_default_graph()

Step 1: Collect Data

In [7]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data' , one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [8]:
trainX = mnist.train.images
trainY = mnist.train.labels

testX = mnist.test.images
testY = mnist.test.labels

In [9]:
for i in [trainX, trainY, testX, testY]:
    i.shape

(55000, 784)

(55000, 10)

(10000, 784)

(10000, 10)

In [10]:
pd.DataFrame(testY).describe()

,0,1,2,3,4,5,6,7,8,9
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.0000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,0.09800,0.113500,0.103200,0.101000,0.0982,0.089200,0.095800,0.102800,0.097400,0.100900
std,0.29733,0.317219,0.304235,0.301344,0.2976,0.285046,0.294331,0.303713,0.296516,0.301211
min,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.00000,1.000000,1.000000,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000,1.000000


Defining parameters

In [20]:
learning_rate = 0.003

n_features = trainX.shape[1]
n_classes = trainY.shape[1]

# No of Batches
batch_size = 100

# No of epochs
training_epoch = 40

# Logs Path and Model Name
logs_path = r"C:\Users\Nivedit\tmp\mnist\DNN\AdamOptimizer\V2"
model_name = "mnist.ckpt"

# Hidden Layers
K = 200
L = 100
M = 60
N = 30

# Define traiing and test keep%
training_keep = 0.75
test_keep = 1

Input Placeholder

In [21]:
with tf.name_scope("Inputs"):
    X = tf.placeholder(dtype = tf.float32, shape=[None, n_features], name = "X-Input")
    Y_actual = tf.placeholder(dtype = tf.float32, shape=[None, n_classes], name = "Y-Input")
    pkeep = tf.placeholder(tf.float32)

**Using RELU as Activation Function across all Layers but NOT on FINAL Layer since we are using SOFTMAX to consider proabilities for each class**

Layer 1

In [22]:
with tf.name_scope("Layer-1"):
    W1 = tf.Variable(tf.truncated_normal([n_features, K], stddev=0.1))
    b1 = tf.Variable(tf.zeros([K]))
    #Y1 = tf.nn.sigmoid(tf.add(tf.matmul(X,W1),b1))
    Y1 = tf.nn.relu(tf.add(tf.matmul(X,W1),b1))

Layer 2

In [23]:
with tf.name_scope("Layer-2"):
    W2 = tf.Variable(tf.truncated_normal([K, L], stddev=0.1))
    b2 = tf.Variable(tf.zeros([L]))
    #Y2 = tf.nn.sigmoid(tf.add(tf.matmul(Y1,W2),b2))
    Y2 = tf.nn.relu(tf.add(tf.matmul(Y1,W2),b2))

Layer 3

In [24]:
with tf.name_scope("Layer-3"):
    W3 = tf.Variable(tf.truncated_normal([L, M], stddev=0.1))
    b3 = tf.Variable(tf.zeros([M]))
    #Y3 = tf.nn.sigmoid(tf.add(tf.matmul(Y2,W3), b3))
    Y3 = tf.nn.relu(tf.add(tf.matmul(Y2,W3), b3))

Layer 4

In [25]:
with tf.name_scope("Layer-4"):
    W4 = tf.Variable(tf.truncated_normal([M,N], stddev=0.1))
    b4 = tf.Variable(tf.zeros(N))
    #Y4 = tf.nn.sigmoid(tf.add(tf.matmul(Y3,W4), b4))
    Y4 = tf.nn.relu(tf.add(tf.matmul(Y3,W4), b4))
    Y4d = tf.nn.dropout(Y4, pkeep)

Prediction as in** FINAL LAYER**

In [26]:
with tf.name_scope("Output"):
    W5 = tf.Variable(tf.truncated_normal([N, n_classes], stddev=0.1))
    b5 = tf.Variable(tf.zeros(n_classes))
    Y_Logit = tf.matmul(Y4d,W5) + b5
    Y_pred = tf.nn.softmax(tf.add(tf.matmul(Y4d,W5), b5))

Loss

In [27]:
with tf.name_scope("Loss"):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=Y_Logit, labels=Y_actual)
    cross_entropy = tf.reduce_mean(cross_entropy)*100

Adam Optimizer

In [28]:
with tf.name_scope("Output"):
    #train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)
    # Using Adam Optimizer
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

Model Accuracy

In [29]:
with tf.name_scope("Accuracy"):
    prediction = tf.argmax(Y_pred, 1, name="Predict")
    check = tf.equal(prediction, tf.argmax(Y_actual, 1))
    accuracy = tf.reduce_mean(tf.cast(check, tf.float32), name="Accuracy")

Loss Logging and Accuracy Logging

In [30]:
training_loss = tf.summary.scalar("Training_loss", cross_entropy)
training_accuray = tf.summary.scalar("Training_accuracy", accuracy)

test_loss = tf.summary.scalar("Test_loss", cross_entropy)
test_accuracy = tf.summary.scalar("Test_accuracy", accuracy)

Saver to save the Graph

In [31]:
saver = tf.train.Saver()

Creating the Graph with batch count of 100

In [32]:
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    # Creating Log Writer Object
    writer = tf.summary.FileWriter(logs_path, graph = tf.get_default_graph())
    
    # Batch Count
    batch_count = int(trainX.shape[0]/batch_size)
    
    # training cycles as epoch
    for epoch in range(training_epoch):
        
        # number of epochs in 1 batch
        for i in range(batch_count):
            batch_x = trainX[i*batch_size : i*batch_size + batch_size]
            batch_y = trainY[i*batch_size : i*batch_size + batch_size]
            
            train_model, acc, loss = sess.run([train_op, training_accuray, training_loss], 
                                              feed_dict = {X:batch_x , Y_actual:batch_y, pkeep:training_keep})
            
            # Log Training Accuracy and Loss
            writer.add_summary(acc, epoch*batch_count + i)
            writer.add_summary(loss, epoch*batch_count + i)
        
        # Log Test Accuracy and Loss
        acc,loss = sess.run([test_accuracy, test_loss], feed_dict = {X:testX, Y_actual:testY, pkeep:test_keep})
        writer.add_summary(acc, epoch*batch_count + batch_count)
        writer.add_summary(loss, epoch*batch_count + batch_count)
        
        if epoch %5 == 0:
            print("Epoch is: ", epoch)
            print("Test Accuracy is: ", accuracy.eval(feed_dict={X:testX, Y_actual:testY, pkeep:test_keep}))
        
    # Save the model
    saver.save(sess, os.path.join(logs_path, model_name))

Epoch is:  0
Test Accuracy is:  0.9499
Epoch is:  5
Test Accuracy is:  0.9752
Epoch is:  10
Test Accuracy is:  0.9774
Epoch is:  15
Test Accuracy is:  0.9787
Epoch is:  20
Test Accuracy is:  0.9731
Epoch is:  25
Test Accuracy is:  0.9785
Epoch is:  30
Test Accuracy is:  0.978
Epoch is:  35
Test Accuracy is:  0.9784


'C:\\Users\\Nivedit\\tmp\\mnist\\DNN\\AdamOptimizer\\V2\\mnist.ckpt'